In [ ]:
!pip install pandas requests matplotlib


In [4]:
import pandas as pd
import requests
import time
import json
import matplotlib.pyplot as plt
from datetime import datetime

In [10]:
# 🔐 Insert your API key here
API_KEY = "e8b43912-6603-413c-b544-3ca7f47cd06b"

# 🧠 Function to format headers
def get_headers(api_key):
    return {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json"
    }


In [17]:
def list_sports_markets(api_key):
    url = "https://api.elections.kalshi.com/trade-api/v2/markets/"
    headers = get_headers(api_key)
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("❌ Error fetching market list:", response.status_code, response.text)
        return []

    markets = response.json().get("markets", [])
    simplified = [(m["ticker"], m["title"]) for m in markets]
    
    print(f"✅ Found {len(simplified)} markets:")
    for ticker, title in simplified:
        print(f"{ticker:40} → {title}")
    
    return simplified

In [7]:
def list_filtered_markets(api_key, keyword="football"):
    url = "https://api.elections.kalshi.com/trade-api/v2/markets/"
    headers = {"Authorization": f"Bearer {api_key}"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("❌ Error fetching market list:", response.status_code, response.text)
        return []

    all_markets = response.json().get("markets", [])
    
    # Filter by keyword in title (case-insensitive)
    filtered = [
        (m["ticker"], m["title"]) for m in all_markets
        if keyword.lower() in m["title"].lower()
    ]
    
    print(f"✅ Found {len(filtered)} markets with keyword '{keyword}':")
    for ticker, title in filtered:
        print(f"{ticker:40} → {title}")
    
    return filtered


In [8]:
def list_liquid_markets(api_key):
    url = "https://api.elections.kalshi.com/trade-api/v2/markets/"
    headers = {"Authorization": f"Bearer {api_key}"}
    response = requests.get(url, headers=headers)

    markets = response.json().get("markets", [])
    filtered = sorted(
        [(m["ticker"], m["title"], m["liquidity_dollars"]) for m in markets if m.get("liquidity")],
        key=lambda x: float(x[2]), reverse=True
    )

    print("Top liquid markets:")
    for ticker, title, liquidity in filtered[:10]:
        print(f"{ticker:40} → ${title}")


In [14]:
list_liquid_markets(API_KEY)

Top liquid markets:


In [13]:
list_filtered_markets(API_KEY, keyword='Atlanta')

✅ Found 0 markets with keyword 'Atlanta':


[]

In [18]:
list_sports_markets(API_KEY)

✅ Found 100 markets:
KXMVENFLMULTIGAMEEXTENDED-S20253068F4C2150-0BBE4A3C8C0 → yes Nico Collins,yes Cooper Kupp,yes Jaxon Smith-Njigba: 110+
KXMVENFLSINGLEGAME-S202571B7D4F6DA9-E6D4323BF26 → yes Dalton Schultz: 20+,yes Cooper Kupp: 30+,yes Kenneth Walker III: 30+
KXMVENFLSINGLEGAME-S20256BEDD65A913-9F7015B0FF4 → yes Nico Collins,yes Jaxon Smith-Njigba,yes C.J. Stroud: 175+,yes Nico Collins: 60+,yes Kenneth Walker III: 40+
KXMVENFLSINGLEGAME-S2025F18FD4E544F-3B313BA6E30 → yes Seattle,yes C.J. Stroud: 175+,yes Sam Darnold: 200+,yes Nico Collins: 50+,yes Woody Marks: 10+,yes Zach Charbonnet: 20+,yes Over 41.5 points scored
KXMVENFLMULTIGAMEEXTENDED-S202530E8E19C4F3-AA5C49E8874 → yes C.J. Stroud: 225+,yes Nico Collins: 70+,yes Xavier Hutchinson: 30+,yes Cooper Kupp: 40+,yes Jaxon Smith-Njigba: 70+
KXMVENFLSINGLEGAME-S202502C33E1A2FB-EF35507F470 → yes Nico Collins,yes Jaxon Smith-Njigba,yes Nico Collins: 60+,yes Jaxon Smith-Njigba: 70+,yes Kenneth Walker III: 40+,yes Seattle wins by over 2.5

[('KXMVENFLMULTIGAMEEXTENDED-S20253068F4C2150-0BBE4A3C8C0',
  'yes Nico Collins,yes Cooper Kupp,yes Jaxon Smith-Njigba: 110+'),
 ('KXMVENFLSINGLEGAME-S202571B7D4F6DA9-E6D4323BF26',
  'yes Dalton Schultz: 20+,yes Cooper Kupp: 30+,yes Kenneth Walker III: 30+'),
 ('KXMVENFLSINGLEGAME-S20256BEDD65A913-9F7015B0FF4',
  'yes Nico Collins,yes Jaxon Smith-Njigba,yes C.J. Stroud: 175+,yes Nico Collins: 60+,yes Kenneth Walker III: 40+'),
 ('KXMVENFLSINGLEGAME-S2025F18FD4E544F-3B313BA6E30',
  'yes Seattle,yes C.J. Stroud: 175+,yes Sam Darnold: 200+,yes Nico Collins: 50+,yes Woody Marks: 10+,yes Zach Charbonnet: 20+,yes Over 41.5 points scored'),
 ('KXMVENFLMULTIGAMEEXTENDED-S202530E8E19C4F3-AA5C49E8874',
  'yes C.J. Stroud: 225+,yes Nico Collins: 70+,yes Xavier Hutchinson: 30+,yes Cooper Kupp: 40+,yes Jaxon Smith-Njigba: 70+'),
 ('KXMVENFLSINGLEGAME-S202502C33E1A2FB-EF35507F470',
  'yes Nico Collins,yes Jaxon Smith-Njigba,yes Nico Collins: 60+,yes Jaxon Smith-Njigba: 70+,yes Kenneth Walker III: 40

In [ ]:
# 🎯 Define your market and date range
MARKET_TICKER = "KXMVENFLMULTIGAMEEXTENDED-S2025B01443386A7-FA3927E4765"  # replace as needed
START_DATE = "2025-09-01T00:00:00Z"
END_DATE   = "2025-11-01T16:00:00Z"


In [ ]:
def fetch_current_orderbook(ticker, api_key):
    url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{ticker}/orderbook/"
    headers = get_headers(api_key)
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("❌ API Error:", response.status_code, response.text)
        return None

    return response.json()


In [ ]:
# Run this to list available markets
markets = list_liquid_markets(API_KEY)

# Paste in your selected ticker from the list above
MARKET_TICKER = "KXMVENFLSINGLEGAME-S2025E182144F026-91F5445947A"

# Fetch live order book data
ob_data = fetch_current_orderbook(MARKET_TICKER, API_KEY)

# Display it
print(json.dumps(ob_data, indent=2))

In [ ]:
def poll_orderbook(ticker, api_key, interval=10, limit=5):
    snapshots = []

    for i in range(limit):
        ob = fetch_current_orderbook(ticker, api_key)
        timestamp = datetime.utcnow().isoformat() + "Z"
        
        if ob:
            ob["timestamp"] = timestamp
            snapshots.append(ob)
            print(f"[{i+1}/{limit}] Snapshot at {timestamp} recorded.")
        else:
            print(f"[{i+1}/{limit}] ❌ Failed to fetch snapshot.")

        time.sleep(interval)

    return snapshots

In [ ]:
snapshots = poll_orderbook(MARKET_TICKER, API_KEY, interval=15, limit=5)

#Flatten and display
df = pd.json_normalize(snapshots)
df["timestamp"] = pd.to_datetime(df["timestamp"])
pd.set_option("display.max_columns", None)
df.head()

In [ ]:
# 🔍 Extract best prices from each snapshot
def get_best_prices(snapshot_row):
    try:
        best_yes = snapshot_row["orderbook.yes_dollars"][0][0] if snapshot_row["orderbook.yes_dollars"] else None
        best_no = snapshot_row["orderbook.no_dollars"][0][0] if snapshot_row["orderbook.no_dollars"] else None
        return pd.Series([best_yes, best_no])
    except Exception as e:
        return pd.Series([None, None])

# 🧠 Apply to all rows
df[["best_yes", "best_no"]] = df.apply(get_best_prices, axis=1)

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(df["timestamp"], df["best_yes"], label="Best YES price ($)", marker='o')
plt.plot(df["timestamp"], df["best_no"], label="Best NO price ($)", marker='x')

plt.title("Market Price Over Time")
plt.xlabel("Timestamp")
plt.ylabel("Price (USD)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.to_csv("orderbook_snapshots.csv", index=False)